In [ ]:
!python --version

Python 3.10.12


In [ ]:
!!python3 -m site

['sys.path = [',
 "    '/content',",
 "    '/env/python',",
 "    '/usr/lib/python310.zip',",
 "    '/usr/lib/python3.10',",
 "    '/usr/lib/python3.10/lib-dynload',",
 "    '/usr/local/lib/python3.10/dist-packages',",
 "    '/usr/lib/python3/dist-packages',",
 ']',
 "USER_BASE: '/root/.local' (exists)",
 "USER_SITE: '/root/.local/lib/python3.10/site-packages' (doesn't exist)",
 'ENABLE_USER_SITE: True']

# Installing Requirements

In [ ]:
!pip install allosaurus

In [ ]:
from google.colab import files
import shutil
import os

#Installing latest model from releases

In [ ]:
#!python -m allosaurus.bin.list_model

Available Models
- uni2005 (default)


In [ ]:
!python -m allosaurus.bin.download_model -m "latest"

downloading model  latest
from:  https://github.com/xinjli/allosaurus/releases/download/v1.0/latest.tar.gz
to:    /usr/local/lib/python3.10/dist-packages/allosaurus/pretrained
please wait...


In [ ]:
!python -m allosaurus.bin.update_phone --lang "deu" --input "/content/phone_dir"

In [ ]:
!python -m allosaurus.bin.list_phone --lang "deu"

a aː b d d̠ d̺ e eː f h i iː j k kʰ l m n oː p pʰ s t tʰ t̠ u uː v x y yː z øː ŋ œ ɐ ɔ ə ɛ ɛː ɡ ɪ ʀ ʁ ʃ ʊ ʏ ʏː ʒ ʔ ʋ ɕ o ɔɪ ç ɑː ɑ r ts p͡f ø tʃ dʒ aɪ aʊ g t͡s ɔʏ


In [ ]:
#!python -m allosaurus.bin.write_phone --lang "deu" --output "phone_dir"

# Preparing train and test data for modelling

In [ ]:
#!gdown 1WQklZVp6WjLbArUnKf1bI64OHxk_ephh
!unzip data.zip

Archive:  data.zip
   creating: data/train/
  inflating: data/train/.DS_Store    
   creating: data/train/clips/
  inflating: data/train/clips/common_voice_de_17566018.wav  
  inflating: data/train/clips/common_voice_de_18640483.wav  
  inflating: data/train/clips/common_voice_de_20733488.wav  
  inflating: data/train/clips/common_voice_de_26681214.wav  
  inflating: data/train/clips/common_voice_de_36484254.wav  
  inflating: data/train/text         
  inflating: data/train/wave         
   creating: data/validate/
  inflating: data/validate/.DS_Store  
   creating: data/validate/clips/
  inflating: data/validate/clips/common_voice_de_18239471.wav  
  inflating: data/validate/clips/common_voice_de_21273725.wav  
  inflating: data/validate/clips/common_voice_de_35040975.wav  
  inflating: data/validate/text      
  inflating: data/validate/wave      


In [ ]:
!python -m allosaurus.bin.prep_feat --model="uni2005" --path="/content/data/validate"

100% 3/3 [00:02<00:00,  1.41it/s]


In [ ]:
!python -m allosaurus.bin.prep_feat --model="uni2005" --path="/content/data/train"

100% 5/5 [00:01<00:00,  2.97it/s]


# Preparing train and test text features

In [ ]:
!python -m allosaurus.bin.prep_token --model="uni2005" --lang="deu" --path="/content/data/train"

100% 5/5 [00:00<00:00, 22286.42it/s]


In [ ]:
!python -m allosaurus.bin.prep_token --model="uni2005" --lang="deu" --path="/content/data/validate"

100% 3/3 [00:00<00:00, 13981.01it/s]


# Training the model

In [ ]:
!python -m allosaurus.bin.adapt_model --pretrained_model="uni2005" --new_model="allo_try2" --path="/content/data" --lang="deu" --device_id=-1 --epoch=5

epoch[batch]: 00[0000] | train loss 4.14317 train per 0.56522
epoch0 | validate per : 0.53039
saving model
epoch[batch]: 01[0000] | train loss 3.92780 train per 0.52609
epoch1 | validate per : 0.53039
saving model
epoch[batch]: 02[0000] | train loss 3.74171 train per 0.52174
epoch2 | validate per : 0.53039
saving model
epoch[batch]: 03[0000] | train loss 3.55150 train per 0.48696
epoch3 | validate per : 0.54696
epoch[batch]: 04[0000] | train loss 3.39371 train per 0.49130
epoch4 | validate per : 0.54144


In [ ]:
src_folder = '/usr/local/lib/python3.10/dist-packages/allosaurus/pretrained'
dest_zip = '/content/pretrained_folder.zip'

shutil.make_archive('/content/pretrained_folder', 'zip', src_folder)
files.download('/content/pretrained_folder.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing the model

In [ ]:
!python -m allosaurus.bin.list_model

Available Models
- uni2005 (default)
- allo_20k_2
- allo_20k


In [ ]:
import allosaurus
from allosaurus.app import read_recognizer
model = read_recognizer("allo_20k_2")
output = model.recognize("/content/Q1200901.wav",'deu')

In [ ]:
output

't͡s v a i'

In [ ]:
output = model.recognize(path_to_voice_file,'deu')

In [ ]:
import numpy as np
from typing import List, Tuple

def levenshtein_distance(reference: List[str], hypothesis: List[str]) -> Tuple[int, int, int]:
    """
    Calculate the Levenshtein distance between two lists of phonemes.
    """
    ref_len = len(reference)
    hyp_len = len(hypothesis)

    # Create a distance matrix
    dist_matrix = np.zeros((ref_len + 1, hyp_len + 1), dtype=int)

    # Initialize the distance matrix
    for i in range(ref_len + 1):
        dist_matrix[i][0] = i
    for j in range(hyp_len + 1):
        dist_matrix[0][j] = j

    # Populate the distance matrix
    for i in range(1, ref_len + 1):
        for j in range(1, hyp_len + 1):
            if reference[i - 1] == hypothesis[j - 1]:
                cost = 0
            else:
                cost = 1
            dist_matrix[i][j] = min(dist_matrix[i - 1][j] + 1,      # Deletion
                                    dist_matrix[i][j - 1] + 1,      # Insertion
                                    dist_matrix[i - 1][j - 1] + cost)  # Substitution

    # The distance is the value in the bottom right corner of the matrix
    distance = dist_matrix[ref_len][hyp_len]

    # Backtrack to find the number of insertions, deletions, and substitutions
    i, j = ref_len, hyp_len
    insertions = deletions = substitutions = 0

    while i > 0 or j > 0:
        if i > 0 and j > 0 and dist_matrix[i][j] == dist_matrix[i - 1][j - 1] + (1 if reference[i - 1] != hypothesis[j - 1] else 0):
            if reference[i - 1] != hypothesis[j - 1]:
                substitutions += 1
            i -= 1
            j -= 1
        elif i > 0 and dist_matrix[i][j] == dist_matrix[i - 1][j] + 1:
            deletions += 1
            i -= 1
        else:
            insertions += 1
            j -= 1

    return insertions, deletions, substitutions

def phoneme_error_rate(reference: List[str], hypothesis: List[str]) -> float:
    """
    Calculate the Phoneme Error Rate (PER).

    :param reference: List of phonemes in the reference transcription.
    :param hypothesis: List of phonemes in the hypothesis transcription.
    :return: Phoneme Error Rate as a float.
    """
    insertions, deletions, substitutions = levenshtein_distance(reference, hypothesis)
    total_errors = insertions + deletions + substitutions
    total_phonemes = len(reference)

    return total_errors / total_phonemes

# Example usage
reference_phonemes = ["p", "h", "o", "n", "e", "m", "e"]
hypothesis_phonemes = ["p", "h", "o", "m", "e", "n", "e"]

per = phoneme_error_rate(reference_phonemes, hypothesis_phonemes)
print(f"Phoneme Error Rate: {per:.2%}")
